## CMBAgent - OSS Testing

*For testing purposes only, to be removed before opening a PR to* `main` .

In [ ]:
# Local model testing supported via https://github.com/acceleratescience/mini-vllm (private Accelerate repo)

import os
from dotenv import load_dotenv

load_dotenv()

BASE_URL = os.getenv("LOCAL_LLM_BASE_URL")
API_KEY = os.getenv("LOCAL_LLM_API_KEY")

local_llm_config = {
    "config_list": [
        {
            "model": "openai/gpt-oss-20b", # Must match model id from endpoint
            "api_key": API_KEY,
            "api_type": "openai",
            "base_url": BASE_URL,
            "price": [0.0, 0.0], # Passed explicitly to ignore warning
        },
    ],
    "cache_seed": None
}

### AG2 ConversableAgent() with `openai/gpt-oss-20b`

Simple chat with a tiny model (`openai/gpt-oss-20b`)to verify ConversableAgent() has OSS

In [ ]:
from autogen import UserProxyAgent, ConversableAgent

# Creates the agent that uses the vLLM LM
assistant = ConversableAgent(
    "agent",
    llm_config=local_llm_config,
    system_message="Answer briefly.",
)

# Create the agent that represents the user in the conversation
user_proxy = UserProxyAgent(
    "user",
    code_execution_config=False,
    system_message="",
    human_input_mode="NEVER",
)

# User asks a question
assistant.initiate_chat(
    user_proxy,
    message="What is the meaning of Denario?",
    max_turns=2,
)

What is the meaning of Denario?

--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
             Model agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
openai/gpt-oss-20b agent 0.00000             86                184           270
**Denario** is a historical silver coin, originally the Roman **denarius** (one‑tenth of a Roman as). It later appeared in medieval and early‑modern Europe (e.g., in Spain and Italy) and is also used to mean “pertaining to the denarius/coin.”

--------------------------------------------------------------------------------


--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'What is the meaning of Denario?', 'role': 'assistant', 'name': 'agent'}, {'content': '', 'role': 'user', 'name': 'user'}, {'content': '**Denario** is a historical silver coin, originally the Roman\u202f**denarius** (one‑tenth of a Roman\u202fas). It later appeared in medieval and early‑modern Europe (e.g., in Spain and Italy) and is also used to mean “pertaining to the denarius/coin.”', 'tool_calls': [], 'role': 'assistant', 'name': 'agent'}, {'content': '', 'role': 'user', 'name': 'user'}], summary='', cost={'usage_including_cached_inference': {'total_cost': 0.0, 'openai/gpt-oss-20b': {'cost': 0.0, 'prompt_tokens': 86, 'completion_tokens': 184, 'total_tokens': 270}}, 'usage_excluding_cached_inference': {'total_cost': 0.0, 'openai/gpt-oss-20b': {'cost': 0.0, 'prompt_tokens': 86, 'completion_tokens': 184, 'total_tokens': 270}}}, human_input=[])

### AG2 ConversableAgent() Tool Calling with OSS

In [ ]:
def add(a: float, b: float) -> float:
    return a + b

assistant.register_for_llm(name="add", description="Add two numbers")(add)
assistant.register_for_execution(name="add")(add)

result = assistant.initiate_chat(
    recipient=assistant,
    message="Use the add tool to add 2 and 3. Then reply with just the number.",
    max_turns=3,
)

for msg in result.chat_history:
    if msg.get("tool_calls"):
        print(msg["tool_calls"])

Use the add tool to add 2 and 3. Then reply with just the number.

--------------------------------------------------------------------------------
             Model agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
openai/gpt-oss-20b agent 0.00000            175                 47           222

--------------------------------------------------------------------------------
5.0

--------------------------------------------------------------------------------
             Model agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
openai/gpt-oss-20b agent 0.00000            426                 92           518
5

--------------------------------------------------------------------------------
             Model agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
openai/gpt-oss-20b agent 0.00000            689                181           870
OK.

--------------------------------------------------------------------------------
             Model agent    